In [38]:
from seleniumwire import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import json
import traceback
from generic_methods import getLocationUrl, getRatingDescription, getRating
import traceback
# pip install webdriver-manager
# pip install selenium-wire
# pip install bs4
# pip install pandas


In [85]:
def getRestaurantList(driver, home_url, NUM_PAGES=1):
    offset = 0
    restaurant_items = []

    for i in range(NUM_PAGES):
        index = url.index('-g')
        offset_url = url[:index] + f"-oa{offset}" + url[index:]
        driver.get(offset_url)
        restaurant_list_page = BeautifulSoup(driver.page_source, "html.parser")
        if i==0:
            restaurant_items.extend(restaurant_list_page.find_all("div", class_="vIjFZ Gi o VOEhq"))
        else:
            restaurant_items.extend(restaurant_list_page.find_all("div", class_="biGQs _P fiohW alXOW NwcxK GzNcM ytVPx UTQMg RnEEZ ngXxk"))

        print(restaurant_items)
        offset += 30


In [82]:
def getRestaurantInfo(page: BeautifulSoup, url):
    restaurant_info = {}
    restaurant_info["url"] = url.get("href")
    restaurant_info["name"] = page.find("h1","HjBfq").contents[0]
    restaurant_info["review_count"] = int(page.find("span","AfQtZ").contents[0].replace(",",""))
    restaurant_info["address"] = page.find("span", "yEWoV").contents[0]
    restaurant_info["rating"] = float(page.find("span", "ZDEqb").contents[0])
    restaurant_info["rating_description"] = getRatingDescription(restaurant_info["rating"])
    # TO ADD
    # website url (requires js)
    # email (requires js)
    # phone number (skill issue)

    #Operating Hours
    operating_hours = []
    try:
        for i ,span in enumerate(page.find("span", "mMkhr").contents[0]):
            if i == 0:
                continue   
            elif span.contains("See all hours"):
                continue
            operating_hours.append(str(span).replace("<span>","").replace("</span>","").replace("<!-- -->","").replace(u'\xa0', u' '))
        
        restaurant_info["operating_hours"] = ",".join(operating_hours)
        print(",".join(operating_hours))
    except Exception as e:
        pass

    try:
        restaurant_info["about"] = page.find("div", "VOzxM").contents[0]
    except:
        pass
    #Tag V1 and V2
    tag = page.find_all("div","CtTod Wf b") + page.find_all("div","tbUiL b") # v1 + v2
    description = page.find_all("div","AGRBq") + page.find_all("div","SrqKb") # v1 + v2
    for i in range(len(tag)):
        restaurant_info[tag[i].contents[0].lower().replace(" ","_")] = description[i].contents[0]

    #Ratings
    ratings = page.find_all("div", "DzMcu")
    for rating in ratings:
        rating_type = rating.find("span", "BPsyj").contents[0]
        classes = rating.find("span", "ui_bubble_rating")
        rating_value = getRating(classes.get("class"))
        restaurant_info[f"rating_{rating_type}"] = rating_value
    
    restaurant_review = []
    for review in page.find_all("div", "reviewSelector"):
        review_info = {}
        review_info["restaurantName"] = restaurant_info["name"]
        review_info["rating"] = getRating(review.find("span", "ui_bubble_rating").get("class"))
        review_info["ratingDate"] = review.find("span", "ratingDate").get("title")
        review_info["title"] = review.find("span", "noQuotes").contents[0]
        review_info["content"] = review.find("p", "partial_entry").contents[0]

        if review.find("span", "stay_date_label"):
            review_info["visitDate"] = review.find("span", "stay_date_label").contents[0]
        
        if review.find("span", "postSnippet"):
            review_info["content"] += review.find("span", "postSnippet").contents[0]
            
        review_info["url"] = review.find("div", "quote").find("a").get("href")
        # print(review_info)
        restaurant_review.append(review_info)
    return restaurant_info, restaurant_review


In [83]:


restaurant_infos = []
restaurant_reviews = []
for restaurant_item in restaurant_items:
    try:
        url = restaurant_item.find("a", class_="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS") # Class of the URL hyperlink
        # print(url.get("href"))
        restaurant_url = "https://www.tripadvisor.com.ph" + url.get("href")
        driver.get(restaurant_url)

        restaurant_info, restaurant_review = getRestaurantInfo( BeautifulSoup(driver.page_source, "html.parser") , url)

        restaurant_infos.append(restaurant_info)
        restaurant_reviews.extend(restaurant_review)
        
    except Exception as e:
        print(f"An error has occured on {restaurant_url}:{traceback.print_exc()}")
        continue


pd.DataFrame(restaurant_infos).to_csv("restaurant_infos.csv", index=False)
pd.DataFrame(restaurant_reviews).to_csv("restaurant_reviews.csv", index=False)
    # break

In [ ]:
print(restaurant_items)

[<div class="vIjFZ Gi o VOEhq" data-test="1_list_item"><div class="qeraN _T qMONr iOIte iJfMg ndRxi CpYrl rcibp FKwyn" style="min-height:212px"><div class="yYtes _T M0 Fl y M0" style="min-height:212px"><div class="MMdJi w"><span><a class="aWhIG _S _Z" href="/Restaurant_Review-g312693-d4049242-Reviews-Lucky_Noodles-Paranaque_Metro_Manila_Luzon.html" target="_blank"><div class="pABFk w _Z Gm A"><div class="WTWEM w _Z"><div class="_T w _Z" data-clicksource="Photo"><div aria-atomic="true" aria-label="Carousel of images" aria-live="polite" class="IdURT w carousel UznXc wSSLS" role="figure"><div class="yMdQy w"><ul class="zHxHb"><li class="CyFNY _A MBoCH"><picture class="NhWcC _R"><img alt="" height="400" loading="lazy" role="none" src="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/ac/0e/fc/lucky-noodles.jpg?w=600&amp;h=400&amp;s=1" srcset="https://dynamic-media-cdn.tripadvisor.com/media/photo-o/14/ac/0e/fc/lucky-noodles.jpg?w=600&amp;h=400&amp;s=1 1x,https://dynamic-media-cdn.t

In [ ]:
print(getLocationUrl("Tagaytay", "Restaurants"))

/Restaurants-g317121-Tagaytay_Cavite_Province_Calabarzon_Region_Luzon.html
